In [3]:
import numpy as np
from scipy import linalg as la

### Exercise 1.

In [26]:
def reduced_QR_decomp(A):
    '''
    A should be an m x n matrix
    '''
    dims = A.shape
    n = dims[1]
    Q = A.copy()
    R = np.zeros((n, n))
    for i in range(n-1):
        R[i, i] = la.norm(Q[:, i])
        Q[:, i] = Q[:, i] / R[i, i]
        
        for j in range(i+1, n-1):
            R[i, j] = Q[:, j] @ Q[:, i]
            Q[:, j] -= R[i, j] * Q[:,i]
    
    return Q, R 

In [27]:
A = np.random.random((6,4))
Q,R = la.qr(A, mode="economic") # Use mode="economic" for reduced QR.
Q

array([[-0.65548522,  0.29745347, -0.25504049,  0.05209052],
       [-0.4715937 , -0.22809548,  0.50437211,  0.54472959],
       [-0.36497521,  0.33275794, -0.48006448,  0.03597978],
       [-0.32978831, -0.01733171,  0.3233467 , -0.8300042 ],
       [-0.32437986, -0.57144071,  0.0429643 , -0.08980914],
       [-0.02736753, -0.64955459, -0.5862612 , -0.04784403]])

In [56]:
Q_hat, R_hat = reduced_QR_decomp(A)
Q_hat.round(5) == -Q.round(5)

array([[ True,  True,  True, False],
       [ True,  True,  True, False],
       [ True,  True,  True, False],
       [ True,  True,  True, False],
       [ True,  True,  True, False],
       [ True,  True,  True, False]], dtype=bool)

In [57]:
R_hat

array([[ 1.29187936,  1.06944085,  1.24597579,  0.        ],
       [ 0.        ,  0.95464771,  0.70190095,  0.        ],
       [ 0.        ,  0.        ,  0.63991656,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [32]:
np.allclose(np.triu(R_hat), R_hat)

True

In [34]:
np.allclose(np.dot(Q.T, Q), np.identity(4))

True

In [43]:
np.allclose(np.dot(Q,R), A)

True

### Exercise 2.

In [64]:
def det_via_QR(A):
    '''
    A must be a square matrix
    '''
    determinante =  np.diag(la.qr(B)[1]).prod()
    return np.abs(determinante)

In [81]:
B = np.random.random((4,4))
np.round(det_via_QR(B), 4) == np.round(la.det(B), 4)

True

### Exercise 3.

In [164]:
def qr_solve(A, b):
    '''
    solving the linear system Ax=b
    '''
    Q, R = la.qr(A)
    y = Q.T @ b
    n = len(B)
    x = np.zeros(n)
    for i in range(1, n+1):
        if i == 1:
            x[n-i] = y[n-i] / R[n-i, n-i]
        else:
            x_vec = R[n-i, n+1-i:] @ x[n+1-i:]
            x[n-i] = (y[n-i] -  x_vec) / R[n-i, n-i]
        
    return x
        
        

In [165]:
b = B @ np.array([1, 2, 3, 4])
qr_solve(B, b)

array([ 1.,  2.,  3.,  4.])

### Exercise 4.

In [247]:
def qr_house(A):
    
    sign = lambda x: 1 if x >= 0 else -1
    m, n = A.shape[0], A.shape[1]
    R = A.copy()
    Q = np.identity(m)
    for k in range(n-1):
        u = R[k:, k].copy()
        u[0] += sign(u[0]) * la.norm(u)
        u = u / la.norm(u)
        R[k:, k:] -= 2 * np.outer(u, u.T @ R[k:, k:])
        Q[k:,:] -= 2 * np.outer(u, u.T @ Q[k:,:])
        
    return Q.T, R

In [248]:
A = np.random.random((5, 3))
Q,R = la.qr(A) # Get the full QR decomposition.
Q_hat, R_hat = qr_house(A)
np.allclose(Q_hat.dot(R_hat), A)

True

### Problem 5.

In [255]:
def hessenberg(A):
    
    sign = lambda x: 1 if x >= 0 else -1
    
    m, n, = A.shape[0], A.shape[1]
    H = A.copy()
    Q = np.identity(m)
    
    for k in range(n-3):
        u = H[k+1:,k].copy()
        u[0] += sign(u[0]) * la.norm(u)
        u = u / la.norm(u)
        H[k+1:,k:] -= 2 * np.outer(u, u.T @ H[k+1:,k:])
        H[:,k+1:] -= 2 * np.outer(H[:,k+1:] @ u, u)
        Q[k+1:,:] -= 2 * np.outer(u, u.T @ Q[k+1:,:])
        
        return H, Q.T

In [256]:
A = np.random.random((8,8))
H, Q = la.hessenberg(A, calc_q=True)

In [257]:
H_hat, Q_hat = hessenberg(A)
np.allclose(np.triu(H_hat, -1), H_hat)
np.allclose(np.dot(np.dot(Q_hat, H_hat), Q_hat.T), A)

True